In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

### Load tsv


In [2]:
exon_file = "mecp2_exons.tsv"
exon_df = pd.read_csv(exon_file, sep="\t")  # Assumes columns: seqname, start, end
exon_df.columns = ['chr', 'start', 'end']
print(exon_df.columns)


Index(['chr', 'start', 'end'], dtype='object')


In [3]:
print(exon_df.head())

    chr      start        end
0  chrX  154094378  154094476
1  chrX  154092184  154092307
2  chrX  154032207  154032557
3  chrX  154021573  154031450
4  chrX  154097604  154097717


### Splice Sites

In [4]:
splice_sites = []
for _, row in exon_df.iterrows():
    splice_sites.append({'chr': row['chr'], 'position': row['end'], 'site_type': 'donor'})
    splice_sites.append({'chr': row['chr'], 'position': row['start'], 'site_type': 'acceptor'})
splice_df = pd.DataFrame(splice_sites)
splice_positions = splice_df['position'].tolist()
donor_positions = splice_df[splice_df['site_type'] == 'donor']['position'].tolist()
acceptor_positions = splice_df[splice_df['site_type'] == 'acceptor']['position'].tolist()
print(splice_df.columns)

Index(['chr', 'position', 'site_type'], dtype='object')


### Load Variant Dataset

In [5]:
file = 'data/aligned_clinvar_result.csv'
df = pd.read_csv(file)

df = df.dropna(subset=["position", "mutated_sequence"])
df["position"] = df["position"].astype(int)
df.columns = df.columns.str.strip()


 ### Region Annotation 

In [6]:
def get_region(pos):
    for _, row in exon_df.iterrows():
        if row['start'] <= pos <= row['end']:
            return 'exon'
    return 'non-exon'

df['region'] = df['position'].apply(get_region)

### Splice Distance

In [7]:
def nearest_distance(pos, positions):
    return min([abs(pos - sp) for sp in positions]) if positions else np.nan

df['donor_distance'] = df['position'].apply(lambda pos: nearest_distance(pos, donor_positions))
df['acceptor_distance'] = df['position'].apply(lambda pos: nearest_distance(pos, acceptor_positions))


In [8]:
print(df.columns)
print(df.head())

Index(['Name', 'Gene(s)', 'Protein change', 'Condition(s)', 'Accession',
       'GRCh37Chromosome', 'GRCh37Location', 'GRCh38Chromosome',
       'GRCh38Location', 'VariationID', 'AlleleID(s)', 'dbSNP ID',
       'Canonical SPDI', 'Variant type', 'Molecular consequence',
       'Germline classification', 'Germline date last evaluated',
       'Germline review status', 'Somatic clinical impact',
       'Somatic clinical impact date last evaluated',
       'Somatic clinical impact review status', 'Oncogenicity classification',
       'Oncogenicity date last evaluated', 'Oncogenicity review status',
       'Unnamed: 24', 'Sequence_ID', 'position', 'Deleted_Sequence',
       'Inserted_Sequence', 'sequence_window', 'mutated_sequence',
       'prev_position_allele', 'next_position_allele', 'aligned_ref',
       'aligned_alt', 'alignment_score', 'mc_synonymous_variant',
       'mc_frameshift_variant', 'mc_3_prime_UTR_variant',
       'mc_5_prime_UTR_variant', 'mc_splice_donor_variant',
       

Calculate Distance to Exon Boundaries

In [9]:
def exon_distances(pos):
    for _, row in exon_df.iterrows():
        if row['start'] <= pos <= row['end']:
            return (abs(pos - row['start']), abs(pos - row['end']))
    return (np.nan, np.nan)
df[['dist_to_exon_start', 'dist_to_exon_end']] = df['position'].apply(lambda pos: pd.Series(exon_distances(pos)))


Nearest Splice Site Type (Donor/Acceptor)

In [10]:
def nearest_splice_type(pos):
    nearest_donor = nearest_distance(pos, donor_positions)
    nearest_acceptor = nearest_distance(pos, acceptor_positions)
    if nearest_donor <= nearest_acceptor:
        return 'donor'
    else:
        return 'acceptor'
df['nearest_splice_type'] = df['position'].apply(nearest_splice_type)


### Sequence Padding

In [11]:
DESIRED_LENGTH = 101
def fix_length(seq, desired_len=DESIRED_LENGTH):
    seq = seq.upper()
    return seq[:desired_len] if len(seq) >= desired_len else seq + 'N' * (desired_len - len(seq))

df['mutated_sequence_fixed'] = df['mutated_sequence'].apply(fix_length)


In [12]:
if 'sequence_window' in df.columns:
    df['sequence_window_fixed'] = df['sequence_window'].apply(fix_length)
else:
    print("⚠️ Warning: 'sequence_window' column not found in the dataset!")


### One-Hot Encoding

In [13]:
base_map = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'N': 4}
NUM_BASES = len(base_map)

def one_hot_encode(seq):
    encoded = [base_map.get(base, 4) for base in seq]
    return to_categorical(encoded, num_classes=NUM_BASES)

# Apply to fixed-length sequences
encoded_sequences = np.array(
    df['mutated_sequence_fixed'].apply(one_hot_encode).tolist()
)


In [14]:
encoded_sequences.shape


(1215, 101, 5)

### Multi-label Targets

In [15]:
label_cols = [col for col in df.columns if col.startswith('mc_')]

# Make sure they’re all integers (0/1)
df[label_cols] = df[label_cols].fillna(0).astype(int)

# Final label matrix
Y = df[label_cols].values


### Binary Label Mapping

In [16]:
df['Germline classification'].unique()


array(['Benign', 'Likely benign', 'Benign/Likely benign',
       'Likely pathogenic', 'Pathogenic', 'Pathogenic/Likely pathogenic'],
      dtype=object)

In [17]:
label_mapping = {
    "Pathogenic": 1, 
    "Likely pathogenic": 1,
    "Pathogenic/Likely pathogenic": 1,
    "Benign": 0, 
    "Likely benign": 0,
    "Benign/Likely benign": 0
}

df["label"] = df["Germline classification"].map(label_mapping).fillna(-1)  

df_variants = df[df["label"] != -1].copy()

df_variants.drop(columns=['Germline classification'], inplace=True)

# Display first few rows
print(df_variants.head())


                               Name Gene(s) Protein change  \
0  NM_001110792.2(MECP2):c.*8503dup   MECP2            NaN   
1  NM_001110792.2(MECP2):c.*8503del   MECP2            NaN   
2  NM_001110792.2(MECP2):c.*7856A>C   MECP2            NaN   
3  NM_001110792.2(MECP2):c.*7748C>T   MECP2            NaN   
4  NM_001110792.2(MECP2):c.*5839C>T   MECP2            NaN   

                 Condition(s)     Accession GRCh37Chromosome  \
0               Rett syndrome  VCV000143289                X   
1               Rett syndrome  VCV000143288                X   
2  not provided|Rett syndrome  VCV000143283                X   
3               Rett syndrome  VCV000143282                X   
4               Rett syndrome  VCV000143280                X   

          GRCh37Location GRCh38Chromosome         GRCh38Location  VariationID  \
0  153287314 - 153287315                X  154021863 - 154021864       143289   
1              153287315                X              154021864       143288   

In [18]:
print(df.columns)

Index(['Name', 'Gene(s)', 'Protein change', 'Condition(s)', 'Accession',
       'GRCh37Chromosome', 'GRCh37Location', 'GRCh38Chromosome',
       'GRCh38Location', 'VariationID', 'AlleleID(s)', 'dbSNP ID',
       'Canonical SPDI', 'Variant type', 'Molecular consequence',
       'Germline classification', 'Germline date last evaluated',
       'Germline review status', 'Somatic clinical impact',
       'Somatic clinical impact date last evaluated',
       'Somatic clinical impact review status', 'Oncogenicity classification',
       'Oncogenicity date last evaluated', 'Oncogenicity review status',
       'Unnamed: 24', 'Sequence_ID', 'position', 'Deleted_Sequence',
       'Inserted_Sequence', 'sequence_window', 'mutated_sequence',
       'prev_position_allele', 'next_position_allele', 'aligned_ref',
       'aligned_alt', 'alignment_score', 'mc_synonymous_variant',
       'mc_frameshift_variant', 'mc_3_prime_UTR_variant',
       'mc_5_prime_UTR_variant', 'mc_splice_donor_variant',
       

### Save Final Preprocessed Data

In [19]:
df.to_csv("data/prep_clinvar_result.csv", index=False)
print("✅ Preprocessing complete. Saved to data/prep_clinvar_result.csv")

✅ Preprocessing complete. Saved to data/prep_clinvar_result.csv
